In [4]:
%pip install --upgrade --quiet langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
pandadata = pd.read_csv('data/review_sample.csv')
pandadata = pandadata.drop('Unnamed: 0', axis=1)
pandadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  1028 non-null   object
 1   text       1028 non-null   object
dtypes: object(2)
memory usage: 16.2+ KB


In [6]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

True

In [12]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [13]:
from langchain_core.messages import HumanMessage

chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to Kannada: I love programming."
        )
    ]
)

AIMessage(content='ನಾನು ಪ್ರೋಗ್ರಾಮಿಂಗ್ ಅನ್ನು ಇಷ್ಟಪಡುತ್ತೇನೆ. (Nānu prōgrāmiṅg annu iṣṭapaḍuttēne)')

In [14]:
# No concept of state:
chat.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content="I'm sorry, I'm not sure what you're referring to. Can you please provide more context or clarify your question?")

In [17]:
from langchain_core.messages import AIMessage

chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to Kannada: I love programming."
        ),
        AIMessage(content='ನಾನು ಪ್ರೋಗ್ರಾಮಿಂಗ್ ಅನ್ನು ಇಷ್ಟಪಡುತ್ತೇನೆ. (Nānu prōgrāmiṅg annu iṣṭapaḍuttēne)'),
        HumanMessage(content="What did you just say?"),
    ]
)

AIMessage(content='I said "ನಾನು ಪ್ರೋಗ್ರಾಮಿಂಗ್ ಅನ್ನು ಇಷ್ಟಪಡುತ್ತೇನೆ" which means "I love programming" in Kannada.')

In [18]:
# define prompt template to make formatting easier, create a chain by piping it into the model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

In [19]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to Kannada: I love programming."
            ),
            AIMessage(content='ನಾನು ಪ್ರೋಗ್ರಾಮಿಂಗ್ ಅನ್ನು ಇಷ್ಟಪಡುತ್ತೇನೆ. (Nānu prōgrāmiṅg annu iṣṭapaḍuttēne)'),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I said "I love programming" in Kannada.')

In [20]:
#in-memory demo message history 
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [21]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."')

In [22]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

AIMessage(content='I said, "J\'adore la programmation," which means "I love programming" in French.')

In [26]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path="data/review_sample.csv", source_column="text")
data = loader.load()

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [28]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# embed and store chunks in vector database
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [29]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("What is a commonly ordered food item at Camellia Grill?")

docs

[Document(page_content='the Camellia Grill.', metadata={'row': 481, 'source': "My two sons both study in New Orleans (one at Loyola University and the other at Tulane).  On one of my first trips to visit them, they made a point of taking me to the Camellia Grill. Boy, am I glad that they did.  What a great place!  The burgers are fabulous, as are the enormous omelettes.  It's fun to sit elbow to elbow at the counter and make new friends of strangers while taking in the floor show that the cooks provide.  I wouldn't dream of visiting NO without at least one trip to the Camellia Grill."}),
 Document(page_content=": 774\nreview_id: 9U7XvPbeqm2FjE2FqocKRw\ntext: Camellia Grill is a little piece of New Orleans history.  Expect great food at low prices.  The portion sizes are\n very large!  You cannot go wrong with a cheeseburger and an order of fries.  It's so good and juicy that I often get it plain....or plain and add an egg.  The omelets and milk shakes are also excellent.  Don't expect 

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain
# modify prompt to accept documents as context

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [28]:
# invode document chain with docs
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("What can you tell me about the Camellia Grill?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

"The Camellia Grill is a popular restaurant in New Orleans known for its milkshakes, omelets, and pies. Customers have raved about the delicious milkshakes, particularly the cherry and orange freezes, as well as the chocolate and coffee friezes. The omelets are also highly recommended, with the eggs whipped up in milkshake mixers for a fluffy result. Additionally, the restaurant is praised for its friendly and personable staff, making customers feel like regulars. It's worth noting that there may be a wait due to the restaurant's popularity, but customers have found it to be worth it."

In [29]:
# creating a retrieval chain
# extract last message from user for input to relevant docs, add to current chain as context
# pass context plus previous messages into the document chain to generate answer

from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [30]:
response = retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

response

{'messages': [HumanMessage(content='What can you tell me about the Camellia Grill?')],
 'context': [Document(page_content=': 989\nreview_id: PsWZU2dyIpWJZYxfgpqZMg\ntext: The Camellia Grill is an institution, so much so that they were filming a segment for a TV show/movie on my recent visit (and the fascination surrounding this outweighed the delay in service that happened as a result).\n\nHere is what you will find...\n\n1. Servers like Michael who take a lot of pride in their work. They call out multiple orders at a time, then the cooks go to work.\n2. 100% low counter seating. Every seat faces the kitchen. If your larger group wants to have a conversation, this may not be the best place.\n3. Delicious breakfast options (standard eggs and bacon, grits, omelettes, pancakes), burgers, shakes, etc. Imagine taking the best of Waffle House and Johnny Rockets and putting it into a fun, local establishment.\n4. It is crowded at peak breakfast and lunch times.', metadata={'source': './data/r

In [31]:
demo_ephemeral_chat_history.add_ai_message(response["answer"])

demo_ephemeral_chat_history.add_user_message("tell me more about that!")

retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    },
)

{'messages': [HumanMessage(content='What can you tell me about the Camellia Grill?'),
  AIMessage(content="The Camellia Grill is a legendary diner in New Orleans known for its vintage feel and classic American menu. It offers delicious breakfast options, burgers, sandwiches, and excellent desserts like pecan pie. The atmosphere is reminiscent of an old-fashioned diner, with servers who wear white dress shirts and bow ties. The seating consists of a low counter facing the kitchen, and the staff is known for being friendly and entertaining. It's a popular spot for locals and tourists alike, and it can get crowded during peak breakfast and lunch times."),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(page_content='As we ate, we chatted and fist-bumped with various servers who came to check on us.  One offered free advice: "It\'s all in ya head," he said sagely, after we commented on the beautiful weather and atmosphere here.  "Ya think positive, ya gonna live 

In [32]:
#define retrieval chain using a pipe directly into the document chain
retrieval_chain_with_only_answer = (
    RunnablePassthrough.assign(
        context=parse_retriever_input | retriever,
    )
    | document_chain
)

retrieval_chain_with_only_answer.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    },
)

"The Camellia Grill is not just a place to eat, it's an experience. It's been a beloved institution in New Orleans for generations, offering a unique blend of delicious comfort food and a warm, nostalgic atmosphere. The classic diner setting with counter seating encourages conversation and a sense of community, and the servers are known for their engaging and friendly interactions with the customers. The food is hearty and comforting, with generous portions and a focus on traditional American favorites like omelets, burgers, and pies. Whether you're visiting for breakfast, brunch, or a late-night snack, the Camellia Grill is a place where you can feel like part of the family."

In [33]:
retriever.invoke("Is the camellia grill kid-friendly?")

[Document(page_content=": 126\nreview_id: ErIdqgbe_TVQuqCWuQQ3BQ\ntext: This place is pretty straight forward. Breakfast and lunch dinner with animated old school wait staff. We usually come on a Saturday after soccer, if the line isn't around the building. Not sure what all the fuss is about, as Camellia Grill isn't the best breakfast stop, but it's decent. The biggest drawback for me is the seating. It's very crowded, and you will definitely be grazing knees or elbows with whomever is sitting next to you. It's fine for a quick bite with a person or two, but not a good place for groups.", metadata={'source': './data/review_sample.csv', 'row': 126}),
 Document(page_content=": 759\nreview_id: NdZhVbRSBMXNQpuAKQXnBQ\ntext: While this is definitely one of the more touristy restaurants in this part of town, Camellia Grill is consistently delicious and a frequent stop for me.\n\nIt has a true diner feel, using only a curved bar with stools rather than tables, which adds to a fun experience 

In [34]:
retriever.invoke("Tell me more about that!")

[Document(page_content='As we ate, we chatted and fist-bumped with various servers who came to check on us.  One offered free advice: "It\'s all in ya head," he said sagely, after we commented on the beautiful weather and atmosphere here.  "Ya think positive, ya gonna live positive."  He continued this speech as a quasi-soliloquy as he went off in search of banana cream pie, and since the only piece left was a little thin, he cut us half a piece of coconut too.  They were both out of this world, and to this moment I regret every bite we left on our plates.\n\nYes, it\'s just a diner.  And no, it\'s not.  It\'s so much more.', metadata={'source': './data/review_sample.csv', 'row': 248}),
 Document(page_content=": 738\nreview_id: ctYtr59egY9tHgxjAuJyQw\ntext: What can I say that hasn't already been said? The place is an institution and I've been going there since I was literally weeks old. Same as it's been for ages, you cram up to the counter when I get my chance, have a phenomenal omel

In [35]:
# add query transformation step that removes references from the input
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# We need a prompt that we can pass into an LLM to generate a transformed search query

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [54]:
# recreate earlier chain with the new query_transforming_retriever_chain

#restart chat history
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

In [55]:
# invoke it
demo_ephemeral_chat_history.add_user_message("what sort of food is Camellia Grill known for?")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response

{'messages': [HumanMessage(content='what sort of food is Camellia Grill known for?'),
  AIMessage(content='Camellia Grill is known for its delicious breakfast options such as eggs, bacon, grits, omelettes, and pancakes, as well as classic American menu items like burgers, sandwiches, shakes, and home-style cooking. The restaurant is also praised for its excellent desserts, including pecan pie.')],
 'context': [Document(page_content=": 1026\nreview_id: UuthR2CkatLYPnUVZMtaTw\ntext: The Camellia Grill is a legend in New Orleans. Definitely the type of place I bring friends and family to when they come to visit. As locals, me and my family go there every time were in the area.  It's my favorite place to eat because the atmosphere, the service, and the food. When we walk in the door I feel like I'm stepping back in time to an old time diner.  The set-up is just one long counter surrounding the grills and it's decorated like were back in the 50's. The servers wear white dress shirts with bo

In [56]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])
response


{'messages': [HumanMessage(content='what sort of food is Camellia Grill known for?'),
  AIMessage(content='Camellia Grill is known for its delicious breakfast options such as eggs, bacon, grits, omelettes, and pancakes, as well as classic American menu items like burgers, sandwiches, shakes, and home-style cooking. The restaurant is also praised for its excellent desserts, including pecan pie.'),
  HumanMessage(content='tell me more about that!'),
  AIMessage(content="Camellia Grill is known for serving classic American comfort food, including breakfast items like eggs, bacon, and pancakes. Their burgers are highly praised for their taste and the variety of dressing options available. The restaurant also offers delicious home-style cooking, as well as exceptional desserts such as apple pie and pecan pie. Overall, it's a place where you can enjoy a wide range of traditional American dishes in a nostalgic diner atmosphere.")],
 'context': [Document(page_content=": 720\nreview_id: bd14Qo_

In [57]:
demo_ephemeral_chat_history.add_user_message("Is it known for anything else other than food?")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='what sort of food is Camellia Grill known for?'),
  AIMessage(content='Camellia Grill is known for its delicious breakfast options such as eggs, bacon, grits, omelettes, and pancakes, as well as classic American menu items like burgers, sandwiches, shakes, and home-style cooking. The restaurant is also praised for its excellent desserts, including pecan pie.'),
  HumanMessage(content='tell me more about that!'),
  AIMessage(content="Camellia Grill is known for serving classic American comfort food, including breakfast items like eggs, bacon, and pancakes. Their burgers are highly praised for their taste and the variety of dressing options available. The restaurant also offers delicious home-style cooking, as well as exceptional desserts such as apple pie and pecan pie. Overall, it's a place where you can enjoy a wide range of traditional American dishes in a nostalgic diner atmosphere."),
  HumanMessage(content='Is it known for anything else other th

In [40]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("When are the best times to come to the Camellia Grill?")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='When are the best times to come to the Camellia Grill?')],
 'context': [Document(page_content=": 1026\nreview_id: UuthR2CkatLYPnUVZMtaTw\ntext: The Camellia Grill is a legend in New Orleans. Definitely the type of place I bring friends and family to when they come to visit. As locals, me and my family go there every time were in the area.  It's my favorite place to eat because the atmosphere, the service, and the food. When we walk in the door I feel like I'm stepping back in time to an old time diner.  The set-up is just one long counter surrounding the grills and it's decorated like were back in the 50's. The servers wear white dress shirts with bow ties like you expect to see in a diner. It has a vintage feel.  It's the time of place that the waiters call out your order to the back using diner lingo and they are always so friendly. If you don't know what to get they are always open to giving suggestions on classic favorites. They are perfect for l